In [ ]:
# 11/23/2021-1
# https://scientific-python.readthedocs.io/en/latest/notebooks_rst/6_Machine_Learning/04_Exercices/02_Practical_Work/02_CNN_1_Beauty.html
from __future__ import absolute_import, division
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import MobileNetV3Small
import os
from os.path import abspath, exists, join
import time

contentPath = os.getcwd()
learningRate = 0.001
batchSize=8

In [ ]:
import tensorflow as tf
baseModel = tf.keras.applications.MobileNetV3Small(
    input_shape=(224, 224, 3), alpha=1.0,
    minimalistic=False, include_top=False,
    weights='imagenet', input_tensor=None,
    classes=1000, pooling=None,
    dropout_rate=0.2, classifier_activation='softmax',
    include_preprocessing=True)
baseModel.name

In [ ]:
def create_model():
    # Create a new neural networks model.
    model = Sequential(name='MobilenetV3small_Sequential')
    
    # Add a pre-trained model, without the last layer
    model.add(MobileNetV3Small(include_top=False,
                               input_shape=(224, 224, 3),
                               pooling='avg',
                               weights='imagenet'))

    # 1st layer as Dense with ReLu
    model.add(Dense(2, activation='relu'))
    # Add a stochastic regularization to avoid overfitting
    model.add(Dropout(rate=0.1))
    # Last layer as Dense for regression
    model.add(Dense(1, activation='linear'))
    # Do not train the first layer model, as it is already pre-trained
    model.layers[0].trainable = False

    # Compile the model with an optimizer
    model.compile(
        loss='mean_squared_error',  # 'mean_absolute_error', 'kullback_leibler_divergence'
        optimizer=Adam(learning_rate=learningRate)
    )
    return model

model = create_model()
model.summary()

In [ ]:
cv2Path=join(contentPath, 'CV2Images')
from tensorflow.keras.utils import image_dataset_from_directory
train_dataset = image_dataset_from_directory(
    cv2Path,
    color_mode='rgb',
    validation_split=0.2,
    subset="training",
    seed=456,
    image_size=(224, 224),
    batch_size=batchSize)
print()

validation_dataset = image_dataset_from_directory(
    cv2Path,
    color_mode='rgb',
    validation_split=0.2,
    subset="validation",
    seed=456,
    image_size=(224, 224),
    batch_size=batchSize)
# clear_output()

In [ ]:
import multiprocessing
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

n_epochs = 20
initial_epoch = 0

callbacks = [EarlyStopping(monitor='val_loss',
                           min_delta=1e-3,
                           patience=10,
                           verbose=1,
                          ),
             ReduceLROnPlateau(monitor='val_loss',
                               factor=0.2,
                               patience=1,
                               cooldown=1,
                               min_lr=0.00001
                              ),
             ModelCheckpoint(
                 filepath='MobilenetV3small_Sequential_{epoch:02d}.h5',
                 monitor='val_loss',
                 save_best_only=True,
                 verbose=1)
            ]

t_start = time.time()
history = model.fit(
    train_dataset,
    epochs=n_epochs,
    callbacks=callbacks,
    initial_epoch=initial_epoch,
    # steps_per_epoch=training_steps_per_epoch,
    validation_data=validation_dataset,
    # validation_steps=validation_steps,
    workers=multiprocessing.cpu_count(),
)

print('Training duration: %.2fs' % float(time.time()-t_start))

In [ ]:
model.name

In [ ]:
import glob
globList=[]
globList=glob.glob('*.h5')
print(len(globList), 'saved epochs')
for item in sorted(globList):
    fullPath=abspath(item)
    if exists(fullPath):
        print(fullPath)
        # os.remove(fullPath)

In [ ]:
# Save / load the trained model
# Load a model from local checkpoints
epoch_id = 20

model_loaded = tf.keras.models.load_model('beauty_model_untuned_epoch%d.h5' % epoch_id)
# Save the final model
models_path = os.path.relpath("models")
assert os.path.exists(models_path)

# Save the final model
model.save(os.path.join(models_path, "beauty_model_untuned"))  # creates a HDF5 file 'final_beauty_model.h5'
# INFO:tensorflow:Assets written to: models/beauty_model_untuned/assets
import os
import tensorflow as tf

# Load the final model
models_path = os.path.relpath("models")
print(models_path)

# model_loaded = tensorflow.keras.models.load_model(os.path.join(models_path, "beauty_model_untuned"))

In [ ]:
# 5. Use your model